# Lifeline

The Lifeline program subsidizes phone and broadband plans for low-income households.
On behalf of the [Federal Communications Commission](https://fcc.gov), Lifeline is administered by the [Universal Service Administrative Company](https://www.usac.org), a nonprofit, through the [Universal Service Fund](https://www.fcc.gov/general/universal-service-fund).

## Eligibility

The FCC provides [at most one Lifeline benefit](https://www.law.cornell.edu/cfr/text/47/54.409#c) to households with income below 135% of the federal poverty guideline, as well as to households with members that participate in Medicaid, SSI, and SNAP (PolicyEngine supports these three), as well as other programs.

## Benefit

Lifeline provides up to \$9.25 per month.

## Examples

### How earnings affect a one-person household's Lifeline benefit

Consider a single person household in California with \$1,000 monthly earned income and \$40 monthly broadband costs.
They would be eligible for Lifeline as their income is below 135% of the poverty line.

In [1]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

sim = IndividualSim(year=2022)
sim.add_person(name="person", employment_income=1000 * 12)
sim.add_spm_unit(members=["person"], broadband_cost=40 * 12)

print("Lifeline: ", sim.calc("lifeline")[0] / 12)

Lifeline:  9.25


What if their earnings change?
They remain eligible based on their income until their income reaches 135% of the poverty line.
However, they remain categorically eligible via SNAP in until their income reaches 200% of the poverty line, California's gross income limit for SNAP.
We can see this by contrasting the Lifeline benefits depending on whether the household claims SNAP.

In [2]:
sim_no_snap = IndividualSim(year=2022)
sim_no_snap.add_person(name="person", employment_income=1000 * 12)
sim_no_snap.add_spm_unit(members=["person"], broadband_cost=40 * 12, snap=0)

sim.vary("employment_income", max=3000 * 12, step=120)
sim_no_snap.vary("employment_income", max=3000 * 12, step=120)

import plotly.express as px

LABELS = dict(
    employment_income="Monthly employment income",
    lifeline="Monthly Lifeline benefit",
    claims_snap="Claims SNAP",
)

df = (
    pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            Yes=sim.calc("lifeline")[0],
            No=sim_no_snap.calc("lifeline")[0],
        )
    )
    / 12
)  # Per month.
df = df.melt(
    id_vars=["employment_income"],
    var_name="claims_snap",
    value_name="lifeline",
)
fig = px.line(
    df,
    "employment_income",
    "lifeline",
    color="claims_snap",
    labels=LABELS,
    title="Lifeline for a one-person household in California",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()